In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD
from animalShelterModule import AnimalShelter 

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"

# db = AnimalShelter(username, password)
db = AnimalShelter(username, password)

# Class read method must support return of list object and accept projection json input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column, and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash. 
# So we remove it in the dataframe here. The df.drop command allows us to drop the column. 
# If we do not set inplace=True, it will return a new dataframe without the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
else:
    print("No '_id' column to drop")
        
## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#######################
# Create pie chart
#######################
def create_pie_chart(df, group_under_100=True):
    
    # prevents overwriting original values when clicking filter buttons to show values under 100
    df = df.copy()
    
    if group_under_100:
        # count breeds
        breed_counts = df['breed'].value_counts()
    
        # group breeds with fewer than 100 dogs
        common_breeds = breed_counts[breed_counts >= 100].index
    
        # replace the grouped breeds with an otherlabel
        df['breed'] = df['breed'].apply(lambda x:x if x in common_breeds else 'Other')
    
    #recount for pie chart
    breed_counts = df['breed'].value_counts().reset_index()
    breed_counts.columns = ['Breed', 'Count']
    fig = px.pie(breed_counts, names='Breed', values='Count', title='Dog Breed Types By Percentage')
    
    fig.update_layout(
        height=800,
        width=800,
        margin=dict(t=40, b=40, l=40, r =40),
        showlegend=True,
        autosize=True,
        title={'x': 0.5},
        paper_bgcolor='black',
        font=dict(color='white') #text
    )
    fig.update_traces(
        textposition='inside',
        textinfo='percent'
    )
    
    return fig

#########################
# Helper for dog filtering
#########################
def get_filtered_df(triggered_id):
    if triggered_id == 'submit-button-one':  # "water rescute" button clicked
        return df[
            (
                (
                    (df['breed'] =="Labrador Retriever Mix") |
                    (df['breed'] =="Chesa Bay Retr Mix") |
                    (df['breed'] =="Newfoundland")
                )& 
                (df['sex_upon_outcome'] == "Intact Female") & 
                (df['age_upon_outcome_in_weeks'] >= 26) & 
                (df['age_upon_outcome_in_weeks'] <= 156) 
            )
        ]
    elif triggered_id == 'submit-button-two':  # "mountain rescue" button clicked
        return df[
            (
                (
                    (df['breed'] =="German Shepherd") |
                    (df['breed'] =="Alaskan Malamute") |
                    (df['breed'] =="Old English Sheepdog") |
                    (df['breed'] =="Siberian Husky") |
                    (df['breed'] =="Rottweiler")
                )& 
                (df['sex_upon_outcome'] == "Intact Male") & 
                (df['age_upon_outcome_in_weeks'] >= 26) & 
                (df['age_upon_outcome_in_weeks'] <= 156) 
            )
        ]
    elif triggered_id == 'submit-button-three':  # "Disaster rescue" button clicked
        return df[
            (
                (
                    (df['breed'] =="Doberman Pinsch") |
                    (df['breed'] =="German Shepherd") |
                    (df['breed'] =="Golden Retriever") |
                    (df['breed'] =="Bloodhound") |
                    (df['breed'] =="Rottweiler")
                )& 
                (df['sex_upon_outcome'] == "Intact Male") & 
                (df['age_upon_outcome_in_weeks'] >= 20) & 
                (df['age_upon_outcome_in_weeks'] <= 300) 
            )
        ]
    else:
       # Default to show all data if no button clicked or Reset button
        return df


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div(
    style={'background': 'black', 'color': 'white'},
    children = [
    #header
    html.Center(html.B(html.H1('Animal-Rescue Training'))),
    # Image at the top
    html.A(
        html.Img(
            src='/assets/pic.png', 
            style={'width': '500px', 'height': 'auto','display':'block', 
                   'margin-left': 'auto', 'margin-right': 'auto'}),
    href='https://www.snhu.edu',
    target="_blank"),
    html.Center(html.B(html.H2('Dashboard by Laurie Sylvester'))),
    html.Center(html.B(html.H6('To get started, pick a filter for the type of rescue dog.'))),
    html.Center(html.B(html.H6('You may also type a value you are looking for into any of the fields.'))),
        html.Hr(),
    
    # Buttons to filter by animal type
    html.Div(
        className='buttonRow',
        style={'display': 'flex', 'justifyContent': 'space-around'},
        children=[
            html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue',
                       style={'backgroundColor':'black', 'color':'deepskyblue', 'border':'1px solid deepsktblue'}),
            html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue',
                       style={'backgroundColor':'black', 'color':'limegreen', 'border':'1px solid limegreen'}),
            html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue',
                       style={'backgroundColor':'black', 'color':'red', 'border':'1px solid red'}),
            html.Button(id='submit-button-four', n_clicks=0, children='Reset Filters',
                       style={'backgroundColor':'black', 'color':'white', 'border':'1px solid solid white'}),
        ]
    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], # Initially select 1st row
        page_action="native",
        page_current=0,
        page_size=10,
        style_header={
            'textAlign': 'center',  # Center align the headers
            'fontWeight': 'bold',  # Make header text bold
            'backgroundColor': 'black',
            'color': 'white',
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',  # Ensures all content is visible
            'backgroundColor': 'black',
            'color': 'white',
        },
        style_table={
            'overflowX': 'auto',  # Ensure horizontal scrolling if needed
            'width': '100%',
        }
    ),
    
    html.Br(),
    html.Hr(),
    
    
    
    # Display the map & pie graph side by side
    
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                className='col s12 m6',
                # add the pie chart
                children=[
                    dcc.Graph(id='pie-chart', figure=create_pie_chart(df))
                ]
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)

    
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # check for empty data frame or invalid index
    if dff.empty or not index or index[0] >= len(dff):
        return [html.Div("Error: No valid row selected or data unavailable.")]

    # Get the coordinates (latitude and longitude)
    latitude = dff.iloc[row]['location_lat']
    longitude = dff.iloc[row]['location_long']
    animal_name = dff.iloc[row]['animal_type']
    breed = dff.iloc[row]['breed']
    age_upon_outcome_in_weeks = dff.iloc[row]['age_upon_outcome_in_weeks']
    sex = dff.iloc[row]['sex_upon_outcome']

    # Return the map with updated marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=12, children=[
            dl.TileLayer(),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(animal_name),
                dl.Popup([
                    html.Div(f"Animal: {animal_name}"),
                    html.Div(f"Breed: {breed}"),
                    html.Div(f"Age (in weeks): {age_upon_outcome_in_weeks}"),
                    html.Div(f"Sex: {sex}")
                ])
            ])
        ])
    ]

# Callback to update the DataTable based on the filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')]
)
def update_table(button1, button2, button3, button4):
    ctx = dash.callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else 'submit-button-four'
    df_filtered = get_filtered_df(triggered_id)
    return df_filtered.to_dict('records')

# Callback to update the map based on the selected row in the DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_on_row_selection(viewData, index):
    # Call the update_map function
    return update_map(viewData, index)

# Callback to update the DataTable based on the filter
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks')]
)
def update_pie_chart(button1, button2, button3, button4):
    ctx = dash.callback_context

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else 'submit-button-four'

    df_filtered = get_filtered_df(triggered_id)
    # Only group under 100 when reset is clicked
    if triggered_id == 'submit-button-four':
        return create_pie_chart(df_filtered, group_under_100=True)
    else:
        return create_pie_chart(df_filtered, group_under_100=False)

app.run_server(debug=True)


Connected successfully.
Success
Dash app running on http://127.0.0.1:17274/
